# Team Assignment 2 - Data Storage and Access

Submitted for Team Trackstars. 

## Notebook Dependencies

In [94]:
import sqlite3 as sql
import pandas as pd
import math

## Helper Functions

In [74]:
def execute(query, params=[]): 
    with sql.connect(db_file) as connection: 
        cursor = connection.execute(query, params) 
        rows = cursor.fetchall()
        return rows

## READ

In [8]:
csv_file = 'data/tips.csv' 
db_file = 'data/tips.db' 

**Create DB from CSV...**

In [25]:
df = pd.read_csv(csv_file)

with sql.connect(db_file) as connection:
    df.to_sql('tips', connection, if_exists='replace', index = False)
    connection.commit()
    print(df.head())

   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


**Validate DB creation, content...** 

In [31]:
with sql.connect(db_file) as connection: 
    cursor = connection.execute("SELECT * FROM tips") 
    rows = cursor.fetchall()
    for row in rows[0:5]: 
        print(row)

(16.99, 1.01, 'Female', 'No', 'Sun', 'Dinner', 2)
(10.34, 1.66, 'Male', 'No', 'Sun', 'Dinner', 3)
(21.01, 3.5, 'Male', 'No', 'Sun', 'Dinner', 3)
(23.68, 3.31, 'Male', 'No', 'Sun', 'Dinner', 2)
(24.59, 3.61, 'Female', 'No', 'Sun', 'Dinner', 4)


**1. Retrieve the average tip percentage for each day of the week**

In [75]:
rows = execute("SELECT AVG(tip),day FROM tips GROUP BY day") 
for row in rows: 
    print(row)

(2.734736842105263, 'Fri')
(2.993103448275862, 'Sat')
(3.255131578947369, 'Sun')
(2.771451612903226, 'Thur')


**2. Find the maximum and minimum total bill amounts**

In [76]:
rows = execute("SELECT MIN(total_bill), MAX(total_bill) FROM tips")
for row in rows:
    print(row)

(3.07, 50.81)


**3. Count the number of parties for each size**

**4. Retrieve the total bill and tip for parties of size 4 or more, where the tip percentage is greater than 15%**

**5. Retrieve the total bill, tip amount, and tip percentage for each combination of day and time, sorted by tip percentage in descending order**

**6. Find the average tip percentage for each combination of day, time, and smoker status**

**7. Retrieve the total bill, tip amount, and tip percentage for each sex, sorted by total bill in descending order, and limit the results to the top 5 records**

**8. Find the maximum and minimum tip percentage for each day and time combination, along with the corresponding total bill and tip amount**

**9. Retrieve the total bill, tip amount, and tip percentage for parties of size 4 or more, where the tip percentage is greater than 15%, and the total bill is between $50 and $100**

**10. Find the average tip percentage for each combination of day, time, and smoker status, but only include combinations with more than 5 records**

**11. Get the unique days reported**

In [77]:
rows = execute("SELECT DISTINCT day FROM tips")
for row in rows: 
    print(row[0]) 

Sun
Sat
Thur
Fri


**12. Group and count the party sizes**

In [105]:
rows = execute("SELECT size, count(*) FROM tips GROUP BY day")
for row in rows: 
    print(row)

(2, 19)
(3, 87)
(2, 76)
(4, 62)


**13. Comparing average tips from smokers and non-smokers**

In [104]:
rows = execute(
"SELECT AVG(tip) FROM tips WHERE smoker='Yes'\
UNION ALL \
SELECT AVG(tip) FROM tips WHERE smoker='No'") 
for row in rows: 
    print(row)

(3.008709677419355,)
(2.9918543046357624,)


**14. TBD query**

**15. TBD query**

## UPDATE

## DELETE